Point Cloud Registration

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg

To gain some insights about the problem, see the fowllowing beautiful link by Russ Tedrake:
https://manipulation.csail.mit.edu/pose.html

In [2]:
def findPoseEstimate(cloudPoints,scenaryPoints):

  # This function estimates the pose of scenary point for a given input
  # cloudPoints,scenaryPoints. We are looking for the translation vector p
  # and the rotation matrix R such that
  # min sum (p + R * cloudPoints_i - scenaryPoints_i)' * (p + R * cloudPoints_i - scenaryPoints_i)
  # Input:
  # a. cloudPoints: np.array of cloud points either in R2 or R3
  # b. scenaryPoints: np.array of scenary points either in R2 or R3

  # Output:
  # i. recoveredPoistion: translation vector (np.array) either in R2 or R3
  # ii. recoveredRotation: rotation matrix either in R2x2 or R3x3

  # We recover the Rotation Matrix and the Translation Vector
  pointDimension = np.shape(cloudPoints)[1];
  numberPoints = np.shape(cloudPoints)[0];

  # Find the mean of cloudPoints and scenaryPoints
  cloudPointsMean = np.mean(cloudPoints,0);
  scenaryPointsMean = np.mean(scenaryPoints,0);

 # Find Data Matrix
  dataMatrix = np.zeros([pointDimension,pointDimension]);
  for i in range(numberPoints):

      scenaryToMeanDiff = scenaryPoints[i] - scenaryPointsMean;
      cloudToMeanDiff = cloudPoints[i] - cloudPointsMean;
      dataMatrix = dataMatrix + np.outer(scenaryToMeanDiff,cloudToMeanDiff);

  # find SVD of dataMatrix
  U, D, VT = linalg.svd(dataMatrix)


  # Solution
  onesVector = np.ones(pointDimension - 1);
  det_UV = linalg.det(U@VT);
  diagVector = np.append(onesVector,det_UV);
  D_Recovered = np.diag(diagVector);
  recoveredRotation = U @ D_Recovered @ VT;
  recoveredPoistion = scenaryPointsMean - recoveredRotation @ cloudPointsMean;

  return recoveredPoistion, recoveredRotation, U, VT, det_UV

In [3]:
# This is a simple test of the algorithm using a bunch of randomly generated points on a circle with perturbed radius

theta = np.arange(0, 360, 30) * np.pi/180;
# np.random.seed(42)
# radiusPerturbation = np.random.random(size = len(theta))
radiusPerturbation = np.array([0.087077, 0.80209, 0.98914, 0.066946, 0.9394, 0.018178, 0.68384, 0.78374, 0.53414, 0.88536, 0.899, 0.62594]);
radius = 5 + 1 *(radiusPerturbation - 0.5);

x = radius * np.cos(theta);
y = radius * np.sin(theta);
cloudPoints = np.vstack((x, y)).T;

## Rotate and Translate point
translationVector = np.array([10,-12]);
thetaRot = 45 * np.pi/180;
R = np.array([[np.cos(thetaRot), -np.sin(thetaRot)],[np.sin(thetaRot), np.cos(thetaRot)]]);

scenaryPoints = translationVector + cloudPoints @ R.T;

recoveredPoistion, recoveredRotation, U, VT, det_UV = findPoseEstimate(cloudPoints,scenaryPoints)
recoveredPoistion


array([ 10., -12.])